In [1]:
# Make lists of the guest smiles strings and the molecule identifiers 
file = 'GDCC_hosts_guests_smiles.txt'

file = open(file, 'r')
text = file.readlines()
file.close()
print("Getting list of molecule IDs...")   
MoleculeIDs = [] #SAMPL6 Molecule ID 
smiles = [] #isomeric SMILES
#Loop over lines and parse
for line in text:
    tmp = line.split(';') #Split into columns
    MoleculeID = tmp[1].strip() #mol ID
    smi = tmp[0] #smiles string
    try:
        MoleculeIDs.append(MoleculeID)
        smiles.append(smi)
    except:
        print("Error storing line: %s" % line)
print("Done!")

Getting list of molecule IDs...
Done!


In [2]:
from openeye.oechem import *
from openeye.oeomega import * # conformer generation

for idx in range( len( smiles ) ):
    #initialize omega, this is a conformation generator
    omega = OEOmega()
    #set the maximum conformer generated to 1
    omega.SetMaxConfs(1) 
    omega.SetIncludeInput(False)
    omega.SetStrictAtomTypes(False) #Leniency is assigning atom types
    omega.SetStrictStereo(True) #Don't generate conformers if stereochemistry not provided
    
    # Generate new OEMol and parse SMILES
    mol = OEMol()
    OEParseSmiles( mol, smiles[idx])
    # Generate one conformer
    status = omega(mol)
    if not status:
        print("Error generating conformers for %s, %s." % (smiles[idx], MoleculeIDs[idx]))

    # Write out PDB of molecule
    ofile = oemolostream('%s.pdb'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

    # Write out MOL2 of molecule
    ofile = oemolostream('%s.mol2'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()
    
    # Write out SDF of molecule
    ofile = oemolostream('%s.sdf'%(MoleculeIDs[idx]))
    OEWriteMolecule(ofile, mol)
    ofile.close()

In [3]:
hosts = ['OA', 'exoOA']
for idx in range( len( hosts ) ):
    inputfile = oemolistream('%s.pdb'%(hosts[idx]) )
    mol = OEMol()
    OEReadMolecule( inputfile, mol )
    inputfile.close()

    # Write to a SDF file
    ofile = oemolostream( '%s.sdf'%(hosts[idx]) ) 
    OEWriteMolecule( ofile, mol)
    ofile.close()

    # Write to a PDB file
    ofile = oemolostream(  '%s.mol2'%(hosts[idx]) )  
    OEWriteMolecule( ofile, mol)
    ofile.close()
    print("Host done:", hosts[idx])

Host done: OA
Host done: exoOA
